In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [2]:
# Importamos el módulo con nuestras funciones
import sys
sys.path.append('./../../')

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

#Own Library modules
import src
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.pca import todoJunto
from src.test_algorithms import comparaModelos
from src.test_algorithms.err_relativo import err_relativo

#### Objetivo:

El objetivo de este notebook es comparar los resultados de PCA de scikit-learn contra:
+ la implementación de PCA usando la función SVD de numpy.
+ la implementación de PCA a partir del algoritmo QR
+ la implementación de PCA a partir del método de la potencia

In [3]:
df = pd.read_csv('../../data/nndb_flat.csv', encoding = "UTF-8")

In [4]:
df.drop(df.columns[df.columns.str.contains('_USRDA')].values, 
        inplace=True, axis=1)
df = df.drop(columns=['ID','FoodGroup','ShortDescrip','Descrip','CommonName','MfgName','ScientificName'])

# Pruebas PCA

In [5]:
####Se estandarizan los datos.
scaler = preprocessing.StandardScaler()
df_nutrientes = scaler.fit_transform(df)

Una vez estandarizados los datos, procedemos a realizar la comparación entre los resultados obtenidos.

Nos apoyamos en las funciones desarrolladas en [comparaModelos.py](https://github.com/123972/PCA-nutricion/blob/master/src/test_algorithms/comparaModelos.py) para presentar los resultados.

## Scikit learn vs PCA a partir de SVD de numpy

In [6]:
df_resultados_sk_np = comparaModelos.compara_resultados_abs_sk_np(df_nutrientes)

df_resultados_sk_np

,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,varianza explicada,True,1.935215e-16,"[0.0, 1.2189963581231018e-16, 1.57089176848327..."
1,valores singulares,True,0.000000e+00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,coeficientes,True,0.000000e+00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,componentes principales,True,1.128716e-10,"[[7.521729907317927e-15, 4.695376192208106e-15..."


Podemos observar que todos los elementos comparados fueron iguales, pues el mayor error relativo, que se presenta en las componentes principales fue del orden de $10^{-10}$.

## Scikit learn vs PCA a partir del algoritmo QR

In [7]:
df_resultados_qr = comparaModelos.compara_resultados_abs_sk_qr(df_nutrientes)
df_resultados_qr

,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,varianza explicada,True,1.912542e-14,"[7.848981292888267e-15, 1.682214974209909e-14,..."
1,eigenvalores,True,2.441949e-14,"[3.259419297349506e-16, 2.4419493161683765e-14..."
2,coeficientes,True,5.478970e-08,"[[2.866773180195502e-14, 4.697647283337805e-14..."
3,componentes principales,True,3.882674e-05,"[[1.9991966332607923e-14, 2.3476880961041193e-..."


Una vez más, todos los elementos comparados fueron iguales. El mayor error relativo, que también ocurre en los componentes principales, es del orden de $10^{-5}$

## Scikit learn vs PCA a partir del método de la potencia

In [8]:
df_resultados_potencia = comparaModelos.compara_resultados_abs_sk_potencia(df_nutrientes)
df_resultados_potencia

,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,varianza explicada,True,3.455962e-15,"[5.857448726036064e-16, 1.3408959939354138e-15..."
1,eigenvalores,False,1.160497e-04,"[0.00011604966925836533, 0.0001160496692602914..."
2,coeficientes,True,1.589268e-12,"[[3.218524490648949e-14, 4.5397431729735165e-1..."
3,componentes principales,True,6.011209e-09,"[[2.4742532589861174e-14, 2.704536686711955e-1..."


A pesar de obtener errores relativos mayores que en los métodos anteriores a nivel de eigenvalores, vemos que finalmente obtenemos una buena aproximación para los componentes principales, donde el mayor error relativo es del orden de $10^{-9}$.

In [9]:
# PCA de scikit learn
pca, var_exp, comp_prin, val_sing, pca_coef, eigenvalues = todoJunto.PCA_from_sklearn(df_nutrientes)

In [10]:
# PCA a partir del método de la potencia
pow_eigenvalues, pow_pca_coef, pow_comp_prin, pow_var_exp = todoJunto.PCA_from_potencia(df_nutrientes)

Al comparar los eigenvalores, comprobamos que las diferencias rondan el $10^{-4}$

In [11]:
np.abs(eigenvalues)-np.abs(pow_eigenvalues)

array([6.32387811e-04, 3.03871226e-04, 2.35801043e-04, 2.18072582e-04,
       1.89819125e-04, 1.32339638e-04, 1.23113295e-04, 1.07491447e-04,
       1.00036923e-04, 9.57045887e-05])